In [1]:
from google.cloud import bigquery
import os

In [2]:
class Config:
    PROJECT_NAME = 'vlille-gcp-etl'
    DATASET_ID = 'vlille_dataset'
    TABLE_ID = 'records'
    GOOGLE_CREDENTIALS = "key-" + PROJECT_NAME + ".json"
    API_URL = ('https://data.lillemetropole.fr/geoserver/wfs?SERVICE=WFS&REQUEST=GetFeature&VERSION=2.0.0'
               '&TYPENAMES=dsp_ilevia%3Avlille_temps_reel&OUTPUTFORMAT=application%2Fjson')
    TIMEZONE = 'Europe/Paris'

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = Config.GOOGLE_CREDENTIALS


In [3]:
# Create a BigQuery client
client = bigquery.Client()

# get the table structure
table = client.get_table(f"{Config.PROJECT_NAME}.{Config.DATASET_ID}.{Config.TABLE_ID}")

# get the table schema
schema = table.schema

# get the table columns
columns = [field.name for field in schema]
print(columns)

['station_id', 'etat', 'nb_velos_dispo', 'nb_places_dispo', 'etat_connexion', 'derniere_maj', 'record_timestamp']


In [7]:
# request to chek data
query = f"""
WITH MaxTimestamp AS (
    SELECT MAX(record_timestamp) AS max_timestamp
    FROM `{Config.PROJECT_NAME}.{Config.DATASET_ID}.{Config.TABLE_ID}`
)
SELECT *
FROM `{Config.PROJECT_NAME}.{Config.DATASET_ID}.{Config.TABLE_ID}`
WHERE record_timestamp = (SELECT max_timestamp FROM MaxTimestamp)
ORDER BY record_timestamp DESC
"""

query_job = client.query(query)
df = query_job.to_dataframe()
df

,station_id,etat,nb_velos_dispo,nb_places_dispo,etat_connexion,derniere_maj,record_timestamp
0,1,RÉFORMÉ,0,0,DÉCONNECTÉ,2022-11-29 10:47:16.181000+00:00,2024-08-17 10:10:01+00:00
1,105,RÉFORMÉ,0,0,DÉCONNECTÉ,2022-11-29 10:47:16.183000+00:00,2024-08-17 10:10:01+00:00
2,106,RÉFORMÉ,0,0,DÉCONNECTÉ,2022-11-29 10:47:16.183000+00:00,2024-08-17 10:10:01+00:00
3,109,RÉFORMÉ,0,0,DÉCONNECTÉ,2022-11-29 10:47:16.184000+00:00,2024-08-17 10:10:01+00:00
4,112,RÉFORMÉ,0,0,DÉCONNECTÉ,2022-11-29 10:47:16.184000+00:00,2024-08-17 10:10:01+00:00
...,...,...,...,...,...,...,...
284,307,EN SERVICE,6,14,CONNECTÉ,2024-08-17 10:09:14.280000+00:00,2024-08-17 10:10:01+00:00
285,308,EN SERVICE,6,14,CONNECTÉ,2024-08-17 10:09:14.281000+00:00,2024-08-17 10:10:01+00:00
286,136,RÉFORMÉ,0,0,DÉCONNECTÉ,2024-04-18 14:46:09.093000+00:00,2024-08-17 10:10:01+00:00
287,140,RÉFORMÉ,0,14,DÉCONNECTÉ,2024-04-18 14:46:09.093000+00:00,2024-08-17 10:10:01+00:00


# repeupler correctement la table des stations ! ils ont peut-être changé les ids ces cons